In [1]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

In [2]:
dog = model['dog']
dog

array([ 5.12695312e-02, -2.23388672e-02, -1.72851562e-01,  1.61132812e-01,
       -8.44726562e-02,  5.73730469e-02,  5.85937500e-02, -8.25195312e-02,
       -1.53808594e-02, -6.34765625e-02,  1.79687500e-01, -4.23828125e-01,
       -2.25830078e-02, -1.66015625e-01, -2.51464844e-02,  1.07421875e-01,
       -1.99218750e-01,  1.59179688e-01, -1.87500000e-01, -1.20117188e-01,
        1.55273438e-01, -9.91210938e-02,  1.42578125e-01, -1.64062500e-01,
       -8.93554688e-02,  2.00195312e-01, -1.49414062e-01,  3.20312500e-01,
        3.28125000e-01,  2.44140625e-02, -9.71679688e-02, -8.20312500e-02,
       -3.63769531e-02, -8.59375000e-02, -9.86328125e-02,  7.78198242e-03,
       -1.34277344e-02,  5.27343750e-02,  1.48437500e-01,  3.33984375e-01,
        1.66015625e-02, -2.12890625e-01, -1.50756836e-02,  5.24902344e-02,
       -1.07421875e-01, -8.88671875e-02,  2.49023438e-01, -7.03125000e-02,
       -1.59912109e-02,  7.56835938e-02, -7.03125000e-02,  1.19140625e-01,
        2.29492188e-01,  

In [3]:
dog.shape

(300,)

In [4]:
puppy = model['puppy']
king = model['king']
queen = model['queen']
book = model['book']
lion = model['lion']
jungle = model['jungle']
tiger = model['tiger']
study = model['study']


In [5]:
#Importing PyMilvus package
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [6]:
#To connect to Milvus
connections.connect("default", host="localhost", port="19530")

In [7]:
# Creating a collection
fields = [
    FieldSchema(name="primarykey", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="words", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=300),
]
schema = CollectionSchema(fields, "Milvus Demo for storing as retrieving embedding")
demo = Collection ("demo", schema)

In [8]:
# Builds indexes on the entities:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128}
}

demo.create_index("embeddings", index)


Status(code=0, message=)

In [9]:
# Insert data in collection
data = [
    [1,2,3,4,5,6,7,8,9], 
    ['puppy', 'dog', 'king', 'queen', 'book', 'lion', 'jungle', 'tiger', 'study'],
    [puppy, dog, king, queen, book, lion, jungle, tiger, study]
]

In [10]:
demo.insert(data)
demo.flush()

In [11]:
demo.load()

In [12]:
result = demo.query(expr="primarykey > 5", output_fields=["primarykey", "words", "embeddings"])

In [13]:
# Sorting the result
sorted_res = sorted(result, key=lambda k: k['words'])
for i in range (0, len(sorted_res)):
    id = sorted_res[i]["primarykey"]
    name = sorted_res[i]["words"]
    print(id, name)

7 jungle
6 lion
9 study
8 tiger


In [14]:
import os


os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'your huggingface token here'

In [15]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [16]:
llm_chain = LLMChain(prompt=prompt,
                     llm=HuggingFaceHub(repo_id="google/flan-t5-large",
                                        model_kwargs={"temperature":0,
                                                      "max_length":256}))
from langchain_core.prompts import PromptTemplate


c:\Users\shiva\miniconda3\envs\cc\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:

title =input("Enter a topic on which you want an paragraph")
template=f"write me a essay based on this title TITLE: {title}"
# print(llm_chain.run(question))
# print(llm_chain.run(question))
# script_template = PromptTemplate(
#     # input_variables = [title], 
#     template='write me a essay based on this title TITLE: What is the capital of France? '
# )
# print(template)

In [ ]:
question = "What area is best for growing wine in France?"

print(llm_chain.run(question))